# Import Library

In [1]:
import numpy as np
import pandas as pd
import random
import math
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn import metrics #akurasi
from sklearn.metrics import confusion_matrix #confussion matrix
from sklearn.metrics import recall_score # recall
from sklearn.metrics import precision_score #precision
from sklearn.metrics import f1_score #f1 score

# Manual Method (Preprocessing and Support)

In [2]:
def count_label(data):
    distinct = [data[0][-1]]
    for i in range(len(data)):
        check = True
        for j in range(len(distinct)):
            if distinct[j] == data[i][-1]:
                check = False
        if check:
            distinct.append(data[i][-1])
    return np.array(distinct).astype(int), len(distinct)

In [3]:
def evaluation(arr_acc):
    mean = np.mean(arr_acc)
    varian = sum((arr_acc - mean) * (arr_acc - mean)) / len(arr_acc)
    std = math.sqrt(varian)
    return mean, std

In [4]:
def handleMissValue(data):
    filled_data = np.array([[0 for i in range(len(data[0]))] for j in range(len(data))])
    cLabel = count_label(data)[1]
    arr_data = [[] for j in range(cLabel)]
    mean = []
    for i in range(len(data)):
        arr_data[int(data[i][-1])-1].append(data[i])
    counter = 0
    for i in range(len(arr_data)):
        mean.append(np.nanmean(arr_data[i], axis=0))
        for j in range(len(arr_data[i])):
            for k in range(len(arr_data[i][j])):
                if np.isnan(arr_data[i][j][k]):
                    arr_data[i][j][k] = mean[i][k]
            filled_data[counter] = arr_data[i][j]
            counter += 1
    return filled_data

In [5]:
def randomize(arr):
    for i in range(len(arr)-1, 0, -1):
        j = random.randint(0, i)
        arr[i], arr[j] = arr[j], arr[i]
    return arr

In [6]:
def normalize(data):
    data = np.array(data).astype(float)
    max_value = np.max(data, axis=0)
    min_value = np.min(data, axis=0)
    for i in range(len(data)):
        for j in range(len(data[0])-1):
            data[i][j] = (data[i][j] - min_value[j]) / (max_value[j] - min_value[j])
    return data

In [7]:
def extraction(file, is_normal = True, is_random = True):
    data = pd.read_excel(file)
    data = handleMissValue(np.array(data))
    data = pd.DataFrame(data)
    data.drop_duplicates()
    data = np.array(data)
    if is_normal:
        data = normalize(data)
    if is_random:
        data = randomize(data.tolist())
    return data

In [8]:
def crossValidation(data, cross_val, cross_index, label_index):
    split_test = int(len(data) * (1 / cross_val))
    change_index = (cross_index - 1) * split_test
    
    for i in range(split_test):
        j = (i + change_index)
        data[i], data[j] = data[j], data[i]
    
    data_train_feature = pd.DataFrame(data[split_test:]).drop([label_index], axis=1)
    data_train_label = pd.DataFrame(pd.DataFrame(data[split_test:]), columns=[label_index]).astype(np.int)
    data_test_feature = pd.DataFrame(data[0:split_test]).drop([label_index], axis=1)
    data_test_label = pd.DataFrame(pd.DataFrame(data[0:split_test]), columns=[label_index]).astype(np.int)
    
    data_train_feature = np.array(data_train_feature)
    data_train_label = np.array(data_train_label)
    data_test_feature = np.array(data_test_feature)
    data_test_label = np.array(data_test_label)

    return data, data_train_feature, data_train_label, data_test_feature, data_test_label

In [9]:
##transform into 1d label array 
def transformArrLabel(X_label, Y_label):
    X_transformed = []
    Y_transformed = []
    for i in range(len(X_label)):
        X_transformed.append(X_label[i][0])
    for i in range(len(Y_label)):
        Y_transformed.append(Y_label[i][0])
    return np.array(X_transformed), np.array(Y_transformed)

# Inisialisasi

In [10]:
k = 15
file = 'Dataset Pegawai.xlsx'
cross_val = 10
label_index = 9 #Kinerja
normalisasi = True
random_data = True
data = extraction(file, normalisasi, random_data)

# Testing

In [11]:
k_range = range(1,16)

arr_coff = [[] for i in range(k)]
arr_mean_acc = [0 for i in range(k)]
arr_std_acc  = [0 for i in range(k)]

for k in k_range:
    arr_acc = [0 for i in range(cross_val)]
    for i in range(1, cross_val + 1):
        data, train_f, train_l, test_f, test_l = crossValidation(data, cross_val, i, label_index)
        train_l, test_l = transformArrLabel(train_l, test_l)
        knn = KNeighborsClassifier(n_neighbors = k, algorithm='brute')
        knn.fit(train_f, train_l)
        y_pred = knn.predict(test_f)
        acc = metrics.accuracy_score(test_l, y_pred)
        arr_acc[i-1] = acc
        
        arr_coff[k-1].append(confusion_matrix(test_l, y_pred))

#         print("Cross-"+str(i)+" Acc:"+ str(acc))
#         print("---------------------------------------------------------")

    mean, std = evaluation(arr_acc)
    arr_mean_acc[k-1] = mean
    arr_std_acc[k-1] = std
#     print('K-{0} Mean : {1} with Standar Deviation : {2}.'.format(k,mean,std))
#     print("==========================================================")

In [12]:
print(np.array(arr_mean_acc))
print(np.array(arr_std_acc))
# print(np.array(arr_coff))

[0.95425532 0.94513678 0.95805471 0.95319149 0.95805471 0.95288754
 0.95759878 0.95410334 0.95592705 0.95075988 0.95258359 0.95106383
 0.95395137 0.94969605 0.95227964]
[0.01119783 0.00664014 0.00838487 0.00661749 0.00649063 0.00880935
 0.0074592  0.00689105 0.00709582 0.00710232 0.00740794 0.00786171
 0.00789835 0.00631934 0.00862922]


In [13]:
def pickCoff(coff):
    choosen_coff = []
    for i in range(len(coff)):
        for j in range(len(coff[i])):
            acc = 0
            for k in range(len(coff[i][j])):
                for l in range(len(coff[i][j][k])):
                    if k == l:
                        acc += coff[i][j][k][l]
            acc /= np.sum(coff[i][j])
            if acc != 1:
                choosen_coff.append(coff[i][j])
                break
            elif j == len(coff[i]) - 1:
                choosen_coff.append(coff[i][j])
    return np.array(choosen_coff)

In [14]:
def splitCoff(picked_coff):
    actual = []
    predict = []
    for i in range(len(picked_coff)):
        predict.append([])
        actual.append([])
        for j in range(len(picked_coff[i])):
            for k in range(len(picked_coff[i][j])):
                for l in range(picked_coff[i][j][k]):
                    if j == k:
                        actual[i].append(j)
                        predict[i].append(j)
                    else:
                        actual[i].append(j)
                        predict[i].append(k)
    return actual, predict

In [16]:
coff_k = 2

picked_coff = pickCoff(np.array(arr_coff))
print(picked_coff[coff_k])

actual, predict = splitCoff(picked_coff)
recall_lib = recall_score(actual[coff_k], predict[coff_k], average=None)
precision_lib = precision_score(actual[coff_k], predict[coff_k], average=None)
f1score_lib = f1_score(actual[coff_k], predict[coff_k], average=None)

print(recall_lib)
print(recall_score(actual[coff_k], predict[coff_k], average='macro'))

print(precision_lib)
print(precision_score(actual[coff_k], predict[coff_k], average='macro'))

print(f1score_lib)
print(f1_score(actual[coff_k], predict[coff_k], average='macro'))

[[243   6   0]
 [ 13 308   0]
 [  0   2  86]]
[0.97590361 0.95950156 0.97727273]
0.9708926331209858
[0.94921875 0.97468354 1.        ]
0.9746340981012658
[0.96237624 0.96703297 0.98850575]
0.9726383172610554
